In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
from IPython.display import display, Markdown
az.style.use('arviz-darkgrid')
import numpy as np
np.random.seed(44)


In [2]:
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams.update({
    'figure.figsize': [8.0, 3.0],
    'figure.facecolor': '#fffff8',
    'axes.facecolor': '#fffff8',
    'figure.constrained_layout.use': True,
    'font.size': 14.0,
    'hist.bins': 'auto',
    'lines.linewidth': 3.0,
    'lines.markeredgewidth': 2.0,
    'lines.markerfacecolor': 'none',
    'lines.markersize': 8.0, 
})

# Normal Hierarchical  Models


Giorgio Corani <br/>
*Bayesian Data Analysis and Probabilistic Programming*
<br/>
<br/>
``giorgio.corani@supsi.ch``





# Based on 

* Alicia A. Johnson, Miles Q. Ott, Mine Dogucu, Bayes Rules! An Introduction to Applied Bayesian Modeling, Chapter 16,  *Hierarchical Models without Predictors*, https://www.bayesrulesbook.com/chapter-16.html


# Modeling songs popularity

* Spotify  provides a data bases of songs and their *popularity* score,  which varies over  the 0-100 scale.  A priori we can expect the average popularity rating to be around 50.


* Higher popularity usually means the song has been played often, especially  recently. 

# Research questions 


*   What’s the typical popularity of a Spotify song?

*   How does popularity vary from artist to artist?


*   For any single artist, how much might popularity vary from song to song?


*  We look for answers with a quantification of the related uncertainty; hence we use probabilistic models.

In [3]:
# the full data set is available from the bayesrule package for R. This is a reduced version which only contains song, author and popularity.
#The data set contains 350 songs by 44 artists
spotify = pd.read_csv("data/spotify.csv")

spotify.head()

,artist,title,popularity
0,Alok,On & On,79
1,Alok,All The Lies,56
2,Alok,Hear Me Now,75
3,Alok,The Wall,65
4,Alok,Hear Me Now,52


In [4]:
spotify["artist"] = spotify["artist"].astype("category")
# artists is a list containing the  name of each artist: [['Alok', 'Atlas Genius', 'Au/Ra', 'BUNT.', 'Beyoncé', ...]
artists = list(spotify["artist"].cat.categories) 


# Hierarchical data set

The data set is hierarchical:

* it comprises  multiple songs for each of 44 artists 
* the artists  were sampled from the population of all artists that have songs on Spotify 

<img src='img/spotify-hierarchical-data-diagram.png' width=600 align="center" >



In [5]:
# Mean popularity of the songs of each artist.
# There are major differences between authors, both in the number of produced songs and in their  popularity.

artist_popularity=spotify.groupby(['artist']).mean().sort_values('popularity').head()
artist_popularity

,popularity
artist,
Mia X,13.250000
Chris Goldarg,16.400000
Soul&Roll,24.200000
Honeywagon,31.666667
Röyksopp,33.250000


In [6]:
# Number of the songs of each artist, which varies between 2 and 40.
artist_count=spotify.groupby(['artist']).count()
artist_count.head()

,title,popularity
artist,,
Alok,19,19
Atlas Genius,4,4
Au/Ra,5,5
BUNT.,3,3
Beyoncé,25,25


# The structure of the data
* The $j$ subscript refers to the $j$-th  artist.

* Each song is written by an artist; artist are indexed by $j$.



*  $y_{ij}$  represents the popularity of the $i$-th song of artist $j$
    * $i \in \{1,2,…,n_j\}$ and $j \in \{1,2,…,44\}$.



* $n_j$ is  the number of total songs composed by  artist $j$ , $j \in  \{1,2,…,44\}$. 



* The sample is the collection of 44 smaller samples (*sub-populations* or *clusters*), one for  each artist.

# Modelling approaches

* *Pooled*: ignore artists and merge the data from all songs.
     * it  does not model differences between artists
     
     
* *Unpooled*:  analyze  independently the population of songs from each artist
    * the prediction of an artist which has published  few songs is very uncertain
    * cannot predict a novel artist


* *Partial pooling*  (hierarchical model):  model the population of  artists and the population of songs from each artist.
    * each artist has his own  estimated popularity
    * the model jointly analyzes  information regarding different artistists
    * it can  predict also artists for which there are no songs

# Pooled model

* The  pooled model ignores the clustering (or *grouping*) structure implied by the different authors.
*  It treats  all songs as a sample from the same population, without modelling the  presence of sub-populations corresponding to the different authors.
* For simplicity,  we assume  the ratings to be normally distributed.

<img src='img/spotify-density.png' width=400 align="center" >

# Pooled model

* We assume the  popularity of every song to be   normally distributed $N(\mu,\sigma)$.  Thus $\mu$ and $\sigma$ are shared by every song;  we refer to them as   *global* parameters since they do not vary by artist.





\begin{align*}
\mu & \sim N(\mu', \sigma_{\mu} ) \\
\sigma & \sim \text{HalfNormal}(\xi)\\
y_{ij} & \sim N(\mu, \sigma) \;\;  \forall i,j 
\end{align*}

# Quiz yourself

* What would be the  prediction of the  pooled model for:

    * a new song of  Mia X, the artist with the lowest mean popularity  (13) ? 
    * Beyoncé, the artist with nearly the highest mean popularity in our sample (70)?
    * Mohsen Beats, a group not present in the sample?




# Answer
* The  complete pooled model ignores the artist-specific information.
* It  predicts the same  popularity for any new song of  every artist, included or not included in the sample.

# The prior on $\mu$

* A priori, we believe $\mu$ to be  around 50; we think that with 95% probability $\mu$ lies in  (10, 90).


* Interpreting this interval as $\mu \pm 2 \sigma$, we set  a standard deviation of (90-50)/2 $\approx$ 20. 

$$ \mu \sim N(50, 20) $$

# The prior on $\sigma$

In [7]:
# A priori, we believe a plausible value of  sigma to be  around 20
# This  would let the popularity vary in the range 50 +- 3*20 (-10 , 110), with a broad support slightly in excess of the allowed range 0-100.
# Using  scale 30, we get the median of the distribution close to 20.
from scipy.stats import halfnorm 
pd.DataFrame(halfnorm.rvs(scale=30, size=10000)).describe()

,0
count,10000.000000
mean,24.089939
std,18.145484
min,0.004981
25%,9.808766
50%,20.401381
75%,34.519428
max,114.781916


# The pooled model


\begin{align*}
\mu    & \sim N(50, 20) \\
\sigma & \sim\text{HalfNormal} (30) \\
y_{ij} | \mu, \sigma & \sim N(\mu, \sigma) \\
\end{align*}

In [8]:
with pm.Model() as pooled_model:
    # prior
    global_mean    =   pm.Normal ('global_mean', 50,  20)
    global_sigma   =  pm.HalfNormal ('global_sigma', sd = 30)

    #likelihood
    popularity          =  pm.Normal ('popularity', mu = global_mean,  sigma = global_sigma, observed = spotify["popularity"])
    trace_pooled     = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [global_sigma, global_mean]


/Users/giorgio/opt/anaconda3/envs/bda/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/giorgio/opt/anaconda3/envs/bda/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/giorgio/opt/anaconda3/envs/bda/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/giorgio/opt/anaconda3/envs/bda/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 16 seconds.


In [9]:
# The posterior summary shows  that Spotify songs have an average popularity of about 58; 
# there is  a relatively large standard deviation from song to song (global_sigma) of about 20 points.
az.summary(trace_pooled)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
global_mean,58.377,1.126,56.237,60.480,0.019,0.013,3594.0,2696.0,1.0
global_sigma,20.708,0.795,19.174,22.143,0.013,0.009,3936.0,2923.0,1.0


# Prediction the popularity of the next song

* The prediction is the same for any artist.



* The trace contains some thousands (e.g., 4000) parameter samples  $<\mu_s, \sigma_s>$. 



* The probabilistic prediction for the next observation is computed  by drawing  a simulated value from each sample:

$$
y_s^{\text{new}} | \mu_s, \sigma_s \; \sim \; N\left(\mu_s, \sigma_s\right)\;\; s=1,2...4000$$



# Posterior prediction

* Each parameter set  $<\mu_i, \sigma_i >$  yields a different prediction:

$$
\left[
\begin{array}{ll} 
\mu_1 &   \sigma_1 \\
\mu_2 & \sigma_2 \\
\vdots & \vdots  \\
\mu_{4000}  & \sigma_{4000} \\
\end{array}
\right]
\;\; \longrightarrow \;\;
\left[
\begin{array}{l} 
y^{\text{new}}_{1} \\
y^{\text{new}}_2 \\
\vdots \\
y^{\text{new}}_{4000} \\
\end{array}
\right]
$$

The resulting collection of 4,000 predictions $Y_{\text{new}}$ approximates the posterior prediction for a new measurement, accounting for two sources of uncertainty:

   * uncertainty in the estimation of $\mu, \sigma$
   * uncertainty in the realization of the noise.

# Implementation of  the posterior prediction 

In [10]:
with pm.Model() as pooled_model:
    # prior
    global_mean    =   pm.Normal ('global_mean', 50,  13)
    global_sigma   =  pm.HalfNormal ('global_sigma', sd = 30)

    #likelihood
    popularity          =  pm.Normal ('popularity', mu = global_mean,  sigma = global_sigma, observed = spotify["popularity"])
    
    #prediction, computed every time a new value of global_mean and global_sigma is sampled.
    prediction          =  pm.Normal('prediction', mu=global_mean, sigma=global_sigma )
    trace_pooled     = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [prediction, global_sigma, global_mean]


/Users/giorgio/opt/anaconda3/envs/bda/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/giorgio/opt/anaconda3/envs/bda/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/giorgio/opt/anaconda3/envs/bda/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 19 seconds.


In [11]:


#with pooled_model:
az.summary(trace_pooled, var_names='prediction')

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
prediction,58.117,21.556,17.368,97.819,0.299,0.221,5198.0,2751.0,1.0


# Unpooled model

* This model independently analyzes the popularity of each artist.


* The $j$-th artist has its own mean popularity $\mu_j$.



* The popularity of the songs produced by artist $j$ are distributed $N(\mu_j, \sigma)$.


* One artist’s mean doesn’t tell us anything about another’s.


* We assume that  $\sigma$ is common to all artist and thus it lacks the  $j$ subscript.


* That simplifies the sampling, but it could be  worth implementing a  variant with a specific $\sigma$ for each artist and comparing the two models via WAIC.
   

# Unpooled model

   * $\mu_j$ = mean song popularity for artist $j$
   * $\sigma$:  standard deviation of popularity from song to song within each artist.


<img src='img/spotify-unpooled.png' width=400 align="center" >

# Unpooled model

* The priors are the same of the pooled model. But every $\mu_j$ is now an independent parameter.

\begin{align*}
\mu_j    & \sim N(50, 20)  \; \; \; j=1,2...44\\
\sigma & \sim \text{HalfNormal}(30) \\
y_{ij}  & \sim N(\mu_j, \sigma^2) \\
\end{align*}

In [12]:
n_artist = len(artist_count) #44

#array of string with the name
artist       = spotify['artist'].values

#We assign a numerical code to each author.
#For each song there is a row in the DataFrame; the field 'artist' contains the numerical code of the artist of that song. 
spotify['artist_code']  = spotify['artist'].astype('category').cat.codes
spotify['artist_code']

0       0
1       0
2       0
3       0
4       0
       ..
345    43
346    43
347    43
348    43
349    43
Name: artist_code, Length: 350, dtype: int8

In [ ]:

with pm.Model() as unpooled_model:
    # We have a vector of means, one for each artist,  each independent from the others.
    mu_artist            =  pm.Normal ('mu_artist', 50,  20, shape= n_artist)
    global_sigma    =  pm.HalfNormal ('global_sigma', sd = 30)

    #likelihood. A different mean for  each artist is inferred.
    #Each value of popularity is normally distributed around the popularity of its artist.
    popularity      =  pm.Normal ('popularity', mu = mu_artist[spotify['artist_code']],  sigma = global_sigma, observed = spotify["popularity"])
    
    trace_unpooled  = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [global_sigma, mu_artist]


In [ ]:
# assing_coords is  necessary to show the artist name
posterior = trace_unpooled.posterior.assign_coords(mu_artist_dim_0=artists)
az.summary(posterior).head()

In [ ]:
#We show only the first 10 artists: we limit and label the graph using the coords option. 
#combined=True implies that we get a single plot for each artist, having merged the samples of the different chains referring to the same artist.

axes = az.plot_forest (trace_unpooled,
                       kind='ridgeplot',
                       hdi_prob=0.68,
                       var_names="mu_artist",
                       combined=True,
                       coords={"mu_artist_dim_0": range(10)},
                       figsize=(8, 6))
artists_10 = artists[:10]
axes[0].set_yticklabels(artists_10[::-1]);

In [ ]:
# MF_ans: if you meant the first 10 artists sorted in some way this is a possibility
trace_unpooled_ = trace_unpooled.copy()
trace_unpooled_.posterior = trace_unpooled.posterior.assign_coords(mu_artist_dim_0=artists)
artist_means = trace_unpooled_.posterior["mu_artist"].mean(("chain", "draw"))
sorted_artists = trace_unpooled_.posterior["mu_artist_dim_0"].sortby(artist_means, ascending=True)
axes = az.plot_forest(trace_unpooled_,
                       kind='ridgeplot',
                       hdi_prob=0.68,
                       var_names="mu_artist",
                       combined=True,
                       figsize=(8,12),
                       coords={"mu_artist_dim_0": sorted_artists[0:20]}
                      )

# Discussion

* The no pooled allows some artists to be more popular than others.



* Yet, it  does not model the *population* of  artists. Even though we have analyzed 44 artist, it is unable to say anything about a novel artist not yet present in the sample.



* Some estimates   are also problematic as even the 68% prediction interval is rather large; those are artist with few published songs.


* For example, our low posterior predictions for Mia X’s next song were based on a measly 4 songs. The other artists’ data suggests that these low ratings might just be a tough break – her next song might be more popular! Similarly, our high posterior predictions for Lil Skies’ next song were based on only 3 songs. In light of the other artists’ data, we might wonder whether this was beginner’s luck that will be tough to maintain.

# Hierarchical model

\begin{equation}
\begin{array}{lrl}
\text{Layer 1:} & \hspace{-0.05in} y_{ij} \sim N( \mu_j, \sigma_y  ) & \hspace{-0.075in} \text{model of how song popularity varies WITHIN artist } j \\
\text{Layer 2:} & \hspace{-0.05in} \mu_j \sim N( \mu, \sigma_\mu) & \hspace{-0.075in}  \text{models the population of artists:  how  $\mu_j$ varies BETWEEN artists}\\
\text{Layer 3:} & \hspace{-0.05in} \text{densities for } \mu, \sigma, \sigma_\mu  & \hspace{-0.075in} \sim \text{prior models for shared global parameters} \\
\end{array}
\end{equation}

# Individual songs for each artist.

* We assume that the popularity of songs  of  artist $j$ to be  normally distributed  $N(μ_j,σ_y)$


*  $σ_y$ is assumed to be the same for all artistist; it expresses the spread in popularity between songs of the same artist


* This is the likelihood of the model:


$$  y_{ij} \sim N(\mu_j, \sigma_y) $$

# Modeling the population of artists


* Here we model the distribution of the $\mu_j$'s.



* The hierarchical model  assumes intead  the 44  artists to be drawn from the population of Spotify artists:


$$ \mu_j \sim N(\mu', \sigma_{\mu}) $$

# Modeling the population of artists

$$ \mu_j \sim N(\mu', \sigma_{\mu}) $$


* $\mu'$ :   average popularity within the population of artists


* $\sigma_\mu$ :  standard deviation of popularity within the population of  artists.


* We want to learn the population of artists, and hence both $\mu'$ and $\sigma_{\mu}$ are  *parameters*, to which we assign a prior.

# Notation alert

* Notice the  difference between $μ_j$ (mean of the j-th artist) and $\mu'$ (mean of the population of artists).


* $\sigma_y$ refers to the standard deviation of $y$ values within each group. 


* $\sigma_{\mu}$ refers to the standard deviation of the means $\mu_j$ within the population of artists.

# Priors for the global parameters $\mu, \sigma_{\mu}, \sigma_y$

\begin{align*}
\mu'           & \sim N(50, 20) & \text{(implies a range of +- 60 around 50)} \\
\sigma_y      & \sim \text{Half Normal}(15) & \text{(discussed in next slide)}     & \\
\sigma_\mu & \sim \text{Half Normal}(6)    & \text{(uncertainty in the location of the global mean)}  \\
\end{align*}

In [ ]:
# how to choose the priors on sigma_y
# We assume the mean popularity to lie in a region of +- 20 points around 50, i.e., a median sigma_y could be 10.
# A possible choice is given below; long tails allow to cover even much larger values for sigma_y than the median.
pd.DataFrame(np.abs(np.random.normal(0, scale=15, size=10000))).describe(percentiles=[0.50, 0.75, 0.95])

In [ ]:
# how to choose the priors on sigma_mu
# We assume the mean popularity to lie in a region of +- 8 points around 50.
pd.DataFrame(np.abs(np.random.normal(0, scale=6, size=10000))).describe(percentiles=[0.50, 0.75, 0.95])

In [ ]:
n_artist = len(artist_count) #44

with pm.Model() as hier_model:
    
    # layer 1: prior on the parameters of the population of artists (hyper-priors). 
    # prior beliefs about the location of the mean popularity within the population of artists
    mu_p          = pm.Normal ('mu_p', 50,  20)
    sigma_mu = pm.HalfNormal ('sigma_mu', 6)
    sigma_y      = pm.HalfNormal ('sigma', 15)

    
    # layer 2: model of the population of artists.
    mu_artist  =  pm.Normal ('mu_artist', mu_p,  sigma_mu, shape= n_artist)
    sigma_y     = pm.HalfNormal ('sigma_y', sd = sigma_y)

    #layer 3 (observation specific for each artist)
    popularity    =  pm.Normal ('popularity', mu = mu_artist[spotify['artist_code']],  sigma = sigma_y, observed = spotify["popularity"])
    
    trace_hier    = pm.sample(return_inferencedata=True)

# Before analyzing the  results, let us think a little bit more..

# Grouping variable or covariate?

* Suppose our dataset includes a categorical variable  $X$


* Should $X$  be modeled  hierarchically or as a categorical covariate?

    *   if the observed data on $X$ covers all categories of interest, it is  better treated as a covariate
    
    *  if the observed categories is a random sample from many possible ones, it is a potential grouping variable.


# Grouping variable or covariate?


* Our data set includes only  44 among thousands of artists on Spotify. Hence, treating artist as a predictor (as in the no pooled model) would limit our understanding to only this small number artists.



* In contrast, treating it as a grouping variable (as in the hierarchical model) allows us to not only learn about the 44 artists in our data, but the broader population of artists from which they were sampled.


# Grouping variable or covariate?

* Consider a daily data set of bike-sharing covering two years; for each day we know whether it was a weekday or a weekend.

* Is the weekend variable a grouping variable or  a covariate?

    * There are only two possible categories: weekend and weekday.
    * The dataset covers both categories, with many observations for each category.
    * Hence  it is a covariate.

# Grouping variable or covariate?

* Consider a data set containing math scores  from a sample of 10 Swiss primary schools. 


* These 10 schools are merely a small sample from the hundreds  of Swiss primary schools.
    * The `school_id`, treated as a covariate (unpooled model) would only allow us to learn about our small sample of schools.
    * Treating it as a grouping variable in a hierarchical model  would allow us to extend our conclusions to the broader population of all schools; this is a better modelling choice.
    
    
* The same consideration applies if you want to analyze e.g. data of a certain disease collected by different hospitals, in which case the involved hospitals can be modelled as a grouping variable.

# Grouping variable or covariate

* In some cases the decision is not clear-cut. For instance you might have a categorical variable with different levels; for some of them many observations are available, for some other levels the number of observations is however very small.

* In this case you might fit both models, and eventually choose according to WAIC.

* WAIC is a model selection criterion, to be discussed more in detail later. For the moment, it is enough to know that we choose we model with lowest WAIC.

# Conceptual exercise: modelling fuel prices

* Define a hierarchical model for the fuel price which considers  different stations located in the same area over a period of e.g. one month.


 * Thus $y_{ij}$ is the price of gas at the $j$-th station in the $i$-th day.


*  Discuss the meaning of all model parameters ( $\mu_j$, $\sigma_y$, $\mu$, $\sigma_{\mu}$).


*  Write the probabilistic model defining the priors based on your experience.    

# Back to the numerical example of Spotify

# Parameters
* The hierarchical Spotify model  has  47 parameters:
    * 44 artist-specific parameters $\mu_j$
    * 3 global parameters $\mu', \sigma_y,  \sigma_\mu\$.

In [ ]:
az.summary(trace_hier, var_names=["mu_p", "sigma_y", "sigma", "mu_artist"])[["mean","hdi_3%","hdi_97%","r_hat"]].head()

In [ ]:
#estimated popularity of each artist, according to the unpooled and to the hierarhical model
# To have a more readable plot, we use a rather short 68% hdi, corresponding to +-1sigma around the mean

trace_unpooled_ = trace_unpooled.copy() # just to avoid touching the existing objects
trace_unpooled_.posterior = trace_unpooled.posterior.assign_coords(mu_artist_dim_0=artists)
trace_hier_ = trace_hier.copy()
trace_hier_.posterior = trace_hier.posterior.assign_coords(mu_artist_dim_0=artists)

axes = az.plot_forest([trace_unpooled_, trace_hier_], 
                      model_names=['unpooled_model', 'hier_model'], combined=True, kind='ridgeplot',
                      var_names='mu_artist', hdi_prob=0.68, coords={"mu_artist_dim_0": artists[0:20]});
 
#mean popularity of all songs
grand_mean = spotify["popularity"].mean()
axes[0].axvline(x=grand_mean);

# Shrinkage: pulling the estimates towards the grand mean

*  The pooled  model ignores the fact that our data is grouped by artist;  the  posterior mean popularity was the same for each artist, which is roughly equivalent to the grand mean (i.e., the mean popularity of the 350 songs)


*  The unpooled model separately analyzes each artist;  its predictive means are roughly equivalent to the sample mean popularity of  each artist.


* In both cases, the estimates are regularized by the prior, hence they are not truly identical to the sample means


* The hierarchical models is a more sophisticated approach. It pulls  (or *shrunk*) the predictions of the unpooled model  toward the global trends of the pooled model.


# Shrinkage

* It the variances $\sigma^2_y$ and  $\sigma^2_{\mu}$ are fixed and known, the posterior mean computed by the hierarchical model for  artist $j$  is: 

$$
\overline{y}_{j,\text{hier}} =
\underbrace{
\frac{\sigma^2_y}{\sigma^2_y + n_j \sigma^2_{\mu}}
}_{\text{weight of the global model}}
\overline{y}_{\text{global}} + 
\underbrace{
\frac{n_j\sigma^2_{\mu}}{\sigma^2_y + n_j \sigma^2_{\mu}}
}_{\text{weight of the local model}}
\overline{y}_j
$$


* This is not our case, as we treat both variances as parameters. Yet, we can use  the above formula to understand *shrinkage*.

* The weight of   local model increases when we have more data ($n_j$) about artist $j$.
* The weights also depend on $\sigma_y$: the larger $\sigma_y$ and hence the variability of popularity for the same artist, the larger the weight of the global model (we cannot trust much the means  $\overline{y}_j$).

# Shrinkage

$$
\overline{y}_{j,\text{hier}} =
\underbrace{
\frac{\sigma^2_y}{\sigma^2_y + n_j \sigma^2_{\mu}}
}_{\text{weight of the global model}}
\overline{y}_{\text{global}} + 
\underbrace{
\frac{n_j\sigma^2_{\mu}}{\sigma^2_y + n_j \sigma^2_{\mu}}
}_{\text{weight of the local model}}
\overline{y}_j
$$

* Scenarios in which the hierarchical posterior predictions shrinks towards the global mean:

*  $n_j$ is low:  we rely more on global trends to understand a group for which we have little data.

* the variability of popularity for songs of the same artist ($\sigma_y$) is large in comparison to the variability the mean populatrity of the artists $\sigma_{\mu}$.

*  the models learns that  artist are very similar to each other ($\sigma_{\mu}$ is low), it increases the weight of the pooled model.

In [ ]:
# showing the amount of shrinkage as a function of the number of songs for each artist

n_j = spotify['artist'].value_counts(sort=False)
mean_unpooled = az.summary(trace_unpooled, var_names=["mu_artist"])["mean"]
mean_hier = az.summary(trace_hier, var_names=["mu_artist"])["mean"]
shrinkage = pd.Series((mean_hier-mean_unpooled).values, index=artists)

df = pd.concat([n_j, shrinkage], axis=1)
df.columns = ["n_j", "shrinkage"]
plt.plot(df["n_j"], df["shrinkage"].abs(), "*")
plt.xlabel("n_j")
plt.ylabel("shrinkage"); 

# Discussion

* The pooled model is very rigid  and  miss the nuances in artists’ mean popularity. 


* No pooled models have the opposite problem. With the built-in flexibility to detect group-specific trends, they tend to have less bias than complete pooled models. But, since they’re tailored to the artists in our sample, it cannot produce predictions regarding  a different set of Spotify artists. Moreover, predictions about artists with limited number of songs have large uncertainty,


* Hierarchical models offer a balanced alternative. Unlike complete pooled models, hierarchical models take group-specific trends into account, and thus will be less biased. And unlike no pooled models, hierarchical models take global trends into account, and thus will be less variable. 

# Probabilistic prediction  with the hierarchical model

* Let us consider e.g. artist n. 25.

* Assuming to access the sample of the trace, our prediction about the popularity of the next song by artist 25 is done simulating, for each sample $s$,: 
 $$y_{25,s}^{new} \sim N( \mu_{25,s}, \sigma_{y,s}) $$
 
 * Implement the prediction and check the consistency of the results by:
     * using the samples from the trace
     * implementing the prediction within the pymc3 model

# Probabilistic prediction  with the hierarchical model


* Write a pseudo-code to predict the popularity of the first  song of a novel artist.


* Implement the prediction

In [ ]:
# prediction of the popularity of a new song for artist 25 by  accessing the trace

a = az.extract_dataset(trace_hier)
y_new_25 = np.random.normal (loc = a["mu_artist"][25],  scale = a["sigma_y"])
pd.DataFrame(y_new_25).describe(percentiles=[0.03,0.97])

# a very few sampled data are negative. Can you trace the reason of this problem and provide a solution?

In [ ]:
n_artist = len(artist_count) #44

with pm.Model() as hier_model_with_preds:
    
    # layer 1: prior on the parameters of the population of artists (hyper-priors). 
    # prior beliefs about the location of the mean popularity within the population of artists
    mu_population          = pm.Normal ('mu_p', 50,  20)
    sigma_mu = pm.HalfNormal ('sigma_mu', 6)
    sigma_y      = pm.HalfNormal ('sigma', 15)

    
    # layer 2: model of the population of artists.
    mu_artist  =  pm.Normal ('mu_artist', mu_population,  sigma_mu, shape= n_artist)
    sigma_y     = pm.HalfNormal ('sigma_y', sd = sigma_y)

    #layer 3 (observation specific for each artist)
    popularity    =  pm.Normal ('popularity', mu = mu_artist[spotify['artist_code']],  sigma = sigma_y, observed = spotify["popularity"])
    
    # == PREDICTIONS
    #prediction of popularity for the next song of  artist 25
    prediction_artist25 = pm.Normal('prediction_artist25', mu = mu_artist[25],  sigma = sigma_y)
    
    #prediction of popularity for a song from a  novel artist. This requires two steps.
    # step 1:  sample the popularity mu_j of the new artist, by sampling from the population of artists.
    mu_new_artist = pm.Normal('new_mean', mu=mu_population, sigma=sigma_mu)
    # step 2:  sample the popularity of a song using the sampled values mu_j for the unknown artist popularity
    prediction_new_artist = pm.Normal('prediction_new_artist', mu = mu_new_artist,  sigma = sigma_y)
    
    trace_hier_with_preds   = pm.sample(return_inferencedata=True)

In [ ]:
# notice the substamtial uncertainty for the prediction of the new artist
az.summary(trace_hier_with_preds, var_names=['prediction_artist25', 'prediction_new_artist'])

# Conceptual exercise 

* Four friends take a  speed-typing tests. Each person repeat 20 times the typing of a certain word.

* Let $y_{ij}$ be the time it takes friend $j$ to complete test $i$.


1) Formalize  a hierarchical model, discussing  the meaning of all model parameters: ($μ_j, μ, \sigma_y, \sigma_{\mu}$)

    
2) Consider the following scenarios and discuss their effect on the posterior distribution of the parameters:

    * The overall results of the 20 timed tests are remarkably similar among the four friends.
    * Each person is quite consistent in their typing times, but there are big differences from person to person – some tend to type much faster than others.

* For each scenario, indicate whether:
* $ σ_y<σ_μ$. 
* $ σ_y≈σ_μ$. 
* $ σ_y>σ_μ$. 